# "Informe del caso práctico del análisis de bicicletas compartidas Cyclistic"
# Ulysses Vidal
# "2023-08-29"

## Introducción

En este informe, realizaremos un análisis exploratorio de datos sobre un conjunto de datos una empresa de bicicletas compartidas de Chicago.

El objetivo es comprender las diferencias entre los patrones de uso de los miembros anuales y los ciclistas ocasionales, para así desarrollar una estrategia de marketing que convierta a los ciclistas ocasionales en miembros anuales.

## Exploración de Datos

Comenzamos cargando y explorando los datos:


In [ ]:
library(readxl)
library(dplyr)
library(tidyverse)  #helps wrangle data
library(lubridate)  #helps wrangle date attributes
library(ggplot2)  #helps visualize data
getwd() #displays your working directory
setwd("~/Casos prácticos Google/Caso práctico Bicicleta - Modificable/Formato XLS Nuevo")

In [ ]:
datos <- read_excel("2022_2023-divvy-tripdata..xlsx")
datos <- read_excel("2022_2023-divvy-tripdata..xlsx", sheet = 1)

head(datos)

A continuación, se haran unas lineas de código con el fin de limpiar el marco de datos considerando:

-   Convertir ride_id y rideable_type en caracteres para que se puedan apilar correctamente.

-   Eliminar los campos de latitud, longitud, año de nacimiento y género, ya que estos datos se eliminaron a partir de 2020.

-   Coherencia en el marco de datos sea coherente con su nomenclatura actual en relación al nombramiento de los ciclistas7.

-   Convertir "ride_length" de Factor a numérico para que podamos ejecutar cálculos sobre los datos.

In [ ]:
# Convert ride_id and rideable_type to character so that they can stack correctly
datos$started_at <- as.character(datos$started_at)
datos$ended_at <- as.character(datos$ended_at)


str(datos)
# Remove lat, long, birthyear, and gender fields as this data was dropped beginning in 2020
datos <- datos %>%  
  select(-c(start_lat, start_lng, end_lat, end_lng,Year,Month))



# Before 2020, Divvy used different labels for these two types of riders ... we will want to make our dataframe consistent with their current nomenclature

# Begin by seeing how many observations fall under each usertype
table(datos$member_casual)


# Reassign to the desired values (we will go with the current 2020 labels)
datos <-  datos %>% 
  mutate(member_casual = recode(member_casual
                                ,"Subscriber" = "member"
                                ,"Customer" = "casual"))

# Check to make sure the proper number of observations were reassigned
table(datos$member_casual)


# Add columns that list the date, month, day, and year of each ride
datos$date <- as.Date(datos$started_at) #The default format is yyyy-mm-dd
datos$month <- format(as.Date(datos$date), "%m")
datos$day <- format(as.Date(datos$date), "%d")
datos$year <- format(as.Date(datos$date), "%Y")
datos$day_of_week <- format(as.Date(datos$date), "%A")


# Add a "ride_length" calculation to all_trips (in seconds)
datos$ride_length <- difftime(datos$ended_at,datos$started_at)

# Convert "ride_length" from Factor to numeric so we can run calculations on the data
is.factor(datos$ride_length)
datos$ride_length <- as.numeric(as.character(datos$ride_length))
is.numeric(datos$ride_length)

A continuación se *eliminan datos "malos"*. El marco de datos incluye algunos cientos de entradas cuando Divvy sacó las bicicletas de los muelles y Divvy verificó su calidad o la longitud de paseo fue negativa Crearemos una nueva versión del marco de datos (v2) ya que se están eliminando datos

In [ ]:
datos_v2 <- datos[!(datos$start_station_name == "HQ QR" | datos$ride_length<0),]

## Análisis Descriptivo

Calculemos algunas estadísticas descriptivas clave para comprender mejor los datos con respecto a la variable relevante que es la duración del viaje.

In [ ]:
# Descriptive analysis on ride_length (all figures in seconds)
mean(datos_v2$ride_length) #straight average (total ride length / rides)
median(datos_v2$ride_length) #midpoint number in the ascending array of ride lengths
max(datos_v2$ride_length) #longest ride
min(datos_v2$ride_length) #shortest ride

Puedes condensar las cuatro líneas anteriores en una línea usando resumen() en el atributo específico:

In [ ]:
summary(datos_v2$ride_length)

Ahora se muestra una comparación estadística entre los miembros con respecto a los usuarios casuales:

In [ ]:
# Compare members and casual users
aggregate(datos_v2$ride_length ~ datos_v2$member_casual, FUN = mean)
aggregate(datos_v2$ride_length ~ datos_v2$member_casual, FUN = median)
aggregate(datos_v2$ride_length ~ datos_v2$member_casual, FUN = max)
aggregate(datos_v2$ride_length ~ datos_v2$member_casual, FUN = min)

A continuación se denota el tiempo promedio de viaje cada día para miembros frente a usuarios ocasionales:

In [ ]:
aggregate(datos_v2$ride_length ~ datos_v2$member_casual + datos_v2$day_of_week, FUN = mean)

## Visualizaciones

A continuación, crearemos algunas visualizaciones para explorar los datos de manera más visual.

In [ ]:
# analyze ridership data by type and weekday
datos_v2 %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>%  #creates weekday field using wday()
  group_by(member_casual, weekday) %>%  #groups by usertype and weekday
  summarise(number_of_rides = n()							#calculates the number of rides and average duration 
            ,average_duration = mean(ride_length)) %>% 		# calculates the average duration
  arrange(member_casual, weekday)								# sorts

1.  *Gráfico de barras* del número de viajes por tipo de ciclista

In [ ]:
# Let's visualize the number of rides by rider type
datos_v2 %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge")+
  labs(title = "Bar graph of number of rides by rider type")

2.  *Gráfico de barras* de duración media del viaje

In [ ]:
# Let's create a visualization for average duration
datos_v2 %>% 
  mutate(weekday = wday(started_at, label = TRUE)) %>% 
  group_by(member_casual, weekday) %>% 
  summarise(number_of_rides = n()
            ,average_duration = mean(ride_length)) %>% 
  arrange(member_casual, weekday)  %>% 
  ggplot(aes(x = weekday, y = average_duration, fill = member_casual)) +
  geom_col(position = "dodge")+
  labs(title = "Bar graph of average trip duration")

## Conclusiones

En base al análisis realizado, se pueden destacar las siguientes conclusiones:

1.  Para el *Gráfico de barras* del número de viajes por tipo de ciclista

-   La distribución de los viajes en bicicletas para un ciclista miembro sugiere que la mayoría de los viajes se encuentran en la mitad de semana en el rango que contempla de [martes, jueves].

-   Al observar el gráfico de barras, se puede notar que donde se presenta el mayor número de viajes en bicicletas para un ciclista miembro son los días miercoles ya que tiene la barra más alta.

-   Al observar el gráfico de barras, se denota un crecimiento en los números de los viajes de los ciclistas casuales desde el lunes hasta el día sabado, pero aún así son menores los viajes que los ciclistas miembros.

-   Los miembros utilizan las bicicletas de manera más constante a lo largo de la semana, mientras que los ciclistas casuales podrían preferir los fines de semana (Sabado, Domingo).

-   Los miembros sean más activos durante los días laborables, mientras que los ciclistas casuales podrían tener un mayor número de viajes los fines de semana.

2.  Para el *Gráfico de barras* de duración media del viaje

-   Al observar el gráfico de barras, se puede notar que donde se presenta la mayor duración media del viaje en bicicletas para un ciclista casual son los días miercoles ya que tiene la barra más alta. En cambio, la mayor duración media del viaje en biciletas para un ciclista miembro lo es el día sabado.

-   Se puede inferir que los miembros tengan viajes más cortos en días laborables debido a su uso regular y predecible de las bicicletas, mientras que los ciclistas casuales podrían tener viajes más largos durante los fines de semana, además de sus días libres.

-   Los ciclistas causales tienden a tener viajes más largos los días miercoles , por lo que podría influir en la programación de mantenimiento o en la distribución de bicicletas.

## Recomendaciones

1.  ***Promociones Estratégicas***: Consiste en ofrecer promociones o descuentos especiales a los ciclistas ocasionales que se inscriban como miembros anuales.
    -   Estas promociones podrían ser limitadas en el tiempo para generar un sentido de urgencia y fomentar la conversión
    -   Estas promociones específicas son con el fin de atraer a más miembros durante los días en el que los ciclistas causales tienen viajes más largos, considerando lo anterior, estos días serían:
        -   Miércoles
        -   Sábado

2.  ***Planificación y asignación de recursos***: Se sabe que la comprensión de los patrones de uso ayudaría a la empresa a asignar recursos de manera más eficiente. Si se sabe que hay un aumento en la demanda de bicicletas en ciertos días, podrían programar el mantenimiento o la redistribución de bicicletas en consecuencia. Lo que sería implicado para los siguientes días:

    -   Lunes
    -   Martes
    -   Miércoles
    -   Jueves
    -   Viernes

    Destacar que estos días son especificamente dada las altas cantidad de viajes realizados por semana.

    Por otro lado, desde el ***punto de vista de la duración promedio en el viaje.*** El día con mayor uso de la bicicleta sería el día miércoles, al presentarse un viaje largo se podría asegurar que haya suficientes bicicletas disponibles en las áreas correspondientes para satisfacer la demanda.

3.  ***Beneficios para Miembros Anuales***: Consiste en destacar claramente los beneficios exclusivos que los miembros anuales reciben, como tarifas reducidas por viaje, acceso prioritario a bicicletas, estacionamiento garantizado, ofertas especiales y más. Mostrar cómo estos beneficios pueden ahorrar dinero y tiempo a largo plazo.

4.  ***Programas de Fidelización***: Referido a implementar un programa de recompensas o puntos para los miembros anuales, donde puedan ganar premios o descuentos adicionales a medida que aumenten su uso y antigüedad en el programa.